# Remarks

* Data normalization
    * Mobilenet expects data from -1 to 1
        * Normalize Input Data or Include in Model
        * TFLite Conversion must fit according to decision
    * Ground Truth Data: for better inspection Data multiplied by 80. Undo the change in the Data Input Pipeline
* Overview in Tutorials:
    * tf.function
    * Repeat addapted Version of using Build in methods for training, ...
    * Save models using keras
        * CaseNet first real model: check implementation of Frey if a Layer needs to be written
        * other Example: depth seperable dilated convolution,

# Libraries

In [1]:
#!for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

import tensorflow as tf
import numpy as np
import os
import time
from datetime import datetime
#import sys
import matplotlib.pyplot as plt
import argparse

import DataProcessing.data_processing as data_processing
import Nets.backbones as backbones
import Nets.features as features
import Nets.losses as losses
import Nets.metrics as metrics
import Nets.visualize as visualize
import Nets.tools as tools

import time


os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#np.set_printoptions(threshold=sys.maxsize)

# Parser

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--model_loaded', type=str, required=False, default=None)
parser.add_argument('--data_base_model', type=str, required=False, default=None)

parser.add_argument('--sigmoid', action='store_true', default=False)
parser.add_argument('--focal', action='store_true', default=False)

parser.add_argument('--beta_upper', type=float, required=False, default=None)
parser.add_argument('--gamma', type=float, required=False, default=None)
parser.add_argument('--alpha', type=float, required=False, default=None)

file_name = None
try:
    file_name = __file__
except:
    print("Jupyter Notebook")
       
if file_name is None:
    args = parser.parse_args("")
    args.train_model = False
    args.cache = True
    args.save = True
    args.sigmoid = False
    args.focal = True
else:    
    args = parser.parse_args()

Jupyter Notebook


# Options

In [3]:
# Generall Parameters
MODEL_LOADED = 'CASENET_FOCAL_LOSS_0.5_g2_a2' if args.model_loaded is None else args.model_loaded
#MODEL_LOADED = 'SGED_FOCAL_LOSS_0.5_g2_a2' if args.model_loaded is None else args.model_loaded
DATA_BASE_MODEL_LOADED = 'SceneNetFloorTiledTextureRandom' if args.data_base_model is None else args.data_base_model

TRAIN_DS = 'Train'
TEST_DS = 'Test'
TEST_HARD_DS = 'Test Hard'
TEST_IMG_DS = 'Test IMG'
HALF = True

# Dataset Loading Parameters
IMG_SIZE_HEIGHT = 1280
IMG_SIZE_WIDTH = 720
NUM_CLASSES = 3
MAX_IMG_TRAIN = 100
MAX_IMG_TEST = 25
SEED = None
NUM_EVAL = 200
NUM_LOOP = 4

# Model Parameters
SAVE = args.save
TRAIN_MODEL = args.train_model


# LOSS
weighted_multi_label_sigmoid_edge_loss = args.sigmoid
focal_loss = args.focal

beta_upper = 0.5 if args.beta_upper is None else args.beta_upper
beta_lower = 1.0 - beta_upper
gamma=2.0 if args.gamma is None else args.gamma 
alpha=2.0 if args.alpha is None else args.alpha
class_weighted = True
weighted_beta=True

THRESHOLD_EDGE_WIDTH_REAL = 1

# Load Dataset, Preprocess Images and Dataset

In [4]:
data = tf.random.uniform(
    [NUM_EVAL, IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH, 3],
    minval=0,
    maxval=255,
    dtype=tf.dtypes.int32,
    seed=None,
    name=None
)

dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.batch(1)

#dataset = dataset.cache()
#dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


paths, files = data_processing.path_definitions(HALF, "", "", "", "", "", "", MODEL_LOADED, DATA_BASE_MODEL_LOADED, make_dirs=False)


2022-06-08 12:37:18.420332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:37:18.428365: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:37:18.428546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 12:37:18.429320: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Model

In [5]:
if weighted_multi_label_sigmoid_edge_loss:
    loss = lambda y_true, y_pred : losses.weighted_multi_label_sigmoid_loss(y_true,y_pred,beta_lower=beta_lower,beta_upper=beta_upper, class_weighted=class_weighted)
elif focal_loss:
    loss = lambda y_true, y_pred : losses.focal_loss_edges(y_true, y_pred, gamma=gamma, alpha=alpha, weighted_beta=weighted_beta,beta_lower=beta_lower,beta_upper=beta_upper, class_weighted=class_weighted)
else:
    raise ValueError("either FocalLoss or WeightedMultiLabelSigmoidLoss must be True")
    

custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracyEdges,
                  "F1Edges": metrics.F1Edges,
                  "<lambda>":loss}

model = tf.keras.models.load_model(paths["MODEL LOADED"], custom_objects=custom_objects)

# evaluate inference time

In [6]:
# Preheat GPU:
i = 0
for d in dataset:
    model.predict(d)

# evaluate:
start = time.perf_counter()
for i in range(NUM_LOOP):
    for d in dataset:
        model.predict(d)
elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)
print('Time per Image {:.4f}'.format(elapsed/NUM_EVAL/NUM_LOOP))
print('Frame per second {:4f}'.format(NUM_EVAL*NUM_LOOP / elapsed))


2022-06-08 12:37:27.742659: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2211840000 exceeds 10% of free system memory.
2022-06-08 12:37:30.688118: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-06-08 12:37:50.636215: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2211840000 exceeds 10% of free system memory.
2022-06-08 12:38:08.181802: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2211840000 exceeds 10% of free system memory.
2022-06-08 12:38:26.490918: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2211840000 exceeds 10% of free system memory.


Elapsed 71.212 seconds.
Time per Image 0.0890
Frame per second 11.234073
